In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Welcome Everyone!

## This is Murat BRONZ, from ENAC (French Civil Aviation School)
## I work on "Flying Vehicles" !

## Contact me : murat.bronz@enac.fr

I am not particularly interseted in measuring GUST or WIND FIELD !

However, these are almost mandatory to have for the applications that I am pushing forward!

Main context why I am here today is (although I have several other related subjects...) different wind measurement techniques that we have been trying.

###  Within the next 10 minutes, I will try to go over an interactive presentation including code sniplets and all the flight data.



In [ ]:
# Let's start by cloning the reprository into Google's server 
# Nothing will be installed into your personal computers !!! Everything is installed and run on hosted computer.

!git clone --recursive https://github.com/mrtbrnz/AVT-347.git
# git submodule update --init --recursive
%cp -r AVT-347/* .
%cd pprz_data
%pip install .
%cd ../

In [ ]:
from pprz_data.pprz_data import DATA

## Autonomous Soaring  for Energy Harvesting (Focusing on Dynamic Soaring these days...)

<img src="content/images/hotliner_laurac_circling_low_re_2.png" width="1200" />

## The vehicle :

<img src="content/images/hotliner_w_probes_lowres.jpg" width="1200"/>

## The whole Setup :

<img src="content/images/system_overview.jpg" width="1200" />

## Flight Altitude

<img src="content/plots/flight_altitude.png" width="1000" />

## How does the signal look like 

<img src="content/plots/noisy_data.png" width="1000" />

## We need to measure :
- Angle of Attack
- Airspeed
- Attitude of the aircraft
- GPS position, and ground velocity
- Optionally the Heading (Magnetic and corrected True North)

##  So that we can calculate the extraced power during the flight pattern

## Obtained results over "One Inclined Circle"
<img src="content/plots/20_11_20__11_28_20_945-969_ma_filter.png" width="900" />

# SmartProbe : 5-Hole Probe with on-board electronics

## Version 0.3
<img src="content/images/smartprobe_v003_pencil_small.png" width="500"/>

## Version 0.4
<img src="content/images/smartprobe_v4_combined.png" width="800"/>

## Calibration Process
<img src="content/images/smartprobe_calibration.jpg" width="800" />

### I will talk a bit more on SmartProbe on the next sections, and you can find more details on the following page (including the flight logs and example processing code)
### Dynamic Soaring Flight Tests - Part-1 : https://mrtbrnz.github.io/dynamic_soaring/


## So SmartProbe is the sensor that is used for wind and airspeed related measurements throughout the dynamic soaring flight tests. 

## However we also would like to obtain "groud truth" 

## Other Measurement Options :

We can think of two extreme cases:
- With the help of expensive and heavy measurement units
- Almost "sensor-less" measurements

## Lidar Equipment

<img src="content/docs/windcube.png" width="500"/>

## Our Setup :

<img src="content/images/windcube_setup.jpg" width="1200" />

<img src="content/docs/wind_reconstruction_1.png" width="800" />
<img src="content/docs/wind_reconstruction_2.png" width="800" />

In [ ]:
# Now lets see some numbers :
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# filename = 'data/lidar/WLS71086_2021_05_04__11_30_09.rtd'
filename = 'data/lidar/WLS71086_2021_04_07__12_16_00.rtd'
data = pd.read_csv(filename , delimiter='\t', skiprows=41, encoding='iso-8859-1')

In [ ]:
data.head(6)

In [ ]:
for column in data.columns:
    print(column)

In [ ]:
# Too lazy to do it automatically...
# altitudes = ['40', '45','50','55','60','65','70','75','80','85','90','95','100','110','120','130','140','150','160','170']
altitudes = [40,45,50,55,60,65,70,75,80,85,90,95,100,110,120,130,140,150,160,170]

In [ ]:
# You can read any value like this :
# data.loc[4,].values[2]
# data.iloc[:,2].values
# Or give a condition to select :
# cond = data.Position == 'V'
# cond = data.Position == '90'
# data[cond].iloc[:,2]

In [ ]:
fig=plt.figure(figsize=(5,10))
for i in range(60):
    wind_array = [data[str(alt)+'m Wind Speed (m/s)'].values[i] for alt in altitudes]
    plt.plot(wind_array,altitudes)
plt.ylabel('Height AGL [m]')
plt.xlabel('Wind Speed [m/s]')

In [ ]:
fig=plt.figure(figsize=(5,10))
cond = data.Position == '0'
for i in range(10):
    wind_array = [data[cond][str(alt)+'m Wind Speed (m/s)'].values[i] for alt in altitudes]
    plt.plot(wind_array,altitudes)
plt.ylabel('Height AGL [m]')
plt.xlabel('Wind Speed [m/s]')

In [ ]:
data[cond]['40m Wind Speed (m/s)'].values[263]

In [ ]:
data[cond]['40m Wind Speed (m/s)'].plot()
# data[cond]['80m Wind Speed (m/s)'].plot()

## Now we have a problem for altitudes below 40m !!!

### We can fly our quadrotors starting from 0m !

In [ ]:
from IPython.display import HTML
from base64 import b64encode
def play_video(file_path, width=1200):
    mp4 = open(file_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""<video width="%s" controls><source src="%s" type="video/mp4"></video>""" % (width, data_url))

In [ ]:
play_video('content/videos/rotating_calib_run.mp4', width=1200)

In [ ]:
play_video('content/videos/student_project_vehicle.mp4', width=1200)

In [ ]:
import scipy as sp
from scipy import signal

def filter_signal(df, freq=0.2):
    m_coeff = 0.0039063
    v_coeff = 0.0000019
    deg_coeff = 0.0139882

    sf = df.index.shape[0]/(df.index.values[-1]-df.index.values[0])
    print('Sampling Frequency :',sf)
    b, a = signal.butter(4, freq/(sf/2), 'low', analog=False)
    padlen = 0
    alt = df.up.values*m_coeff
    phi = signal.filtfilt(b, a, df.phi, padlen=padlen)*deg_coeff
    theta = signal.filtfilt(b, a, df.theta, padlen=padlen)*deg_coeff # method="gust")
    psi = df.psi.values*deg_coeff
    # psi = np.unwrap(df.psi*deg_coeff)
    # psi = signal.filtfilt(b, a, psi, padlen=padlen)*deg_coeff # method="gust")
    return phi, theta, psi, alt

def smooth_signal(sig, sf=50., freq=0.2):
    b, a = signal.butter(4, freq/(sf/2), 'low', analog=False)
    padlen = 0
    return signal.filtfilt(b, a, sig, padlen=padlen)


In [ ]:
ac_id = '4'
filename = 'data/quad/21_05_07__14_29_08.data' # in front of Windshape from 0-11.9-0m/s
flight_1 = DATA(filename, ac_id, data_type='flight', sample_period=0.02)
df1 = flight_1.df_All.copy()

In [ ]:
window=10
df1[['phi','theta']][100:].rolling(window=window).mean().plot(figsize=(20,4)) ; plt.grid()
# df1[['up']][100:].rolling(window=window).mean().plot(figsize=(20,4)); plt.grid()

In [ ]:
phi, theta, psi, alt = filter_signal(df1[50:500])

In [ ]:
fig=plt.figure(figsize=(20,4))
plt.plot(phi)
plt.plot(theta)
# plt.plot(psi)
plt.grid();plt.xlabel('Time [s]');plt.ylabel('Phi & Theta [deg]')

In [ ]:
fig=plt.figure(figsize=(20,4))
total_inclination = np.sqrt(phi**2+theta**2)
total_inclination = smooth_signal(total_inclination, freq=10.) #0.04
plt.plot(total_inclination) # *0.6 to approximate airspeed
plt.grid();plt.xlabel('Time [s]');plt.ylabel('Total inclination [deg]')

In [ ]:
ac_id = "11"
filename = 'data/quad/21_05_04__13_31_43.data'
flight_1 = DATA(filename, ac_id, data_type='flight', sample_period=0.02)
df1 = flight_1.df_All.copy()

In [ ]:
df1.columns

In [ ]:
window=10
df1[['phi','theta']][100:].rolling(window=window).mean().plot(figsize=(20,4)) ; plt.grid()
df1[['up']][100:].rolling(window=window).mean().plot(figsize=(20,4)); plt.grid()

In [ ]:
phi, theta, psi, alt = filter_signal(df1[50:500], freq=0.2)

In [ ]:
fig=plt.figure(figsize=(20,4))
plt.plot(phi)
plt.plot(theta)
# plt.plot(psi)
plt.grid();plt.xlabel('Time [s]');plt.ylabel('Phi & Theta [deg]')

In [ ]:
fig=plt.figure(figsize=(20,4))
total_inclination = np.sqrt(phi**2+theta**2)
total_inclination = smooth_signal(total_inclination, freq=0.02)
plt.plot(total_inclination)
plt.grid();plt.xlabel('Time [s]');plt.ylabel('Total inclination [deg]')

In [ ]:
fig=plt.figure(figsize=(10,15))
high=13000
plt.plot(total_inclination[:high],alt[:high], label='Climbing')
plt.plot(total_inclination[high:],alt[high:], label='Descending')
# plt.plot(total_inclination,alt)
# plt.plot(psi)
plt.grid();plt.xlabel('Total Inclination [deg]');plt.ylabel('Height AGL [m]');plt.legend()

In [ ]:
play_video('content/videos/VKS.mp4', width=1200)

## Open Questions :

In [ ]:
# fig = plt.figure(figsize=(15,10))
# plt.plot(data1_v3.seconds.values, data1_v3.V_avg.values)
# plt.subplot(151)
# plt.plot(data.values, data.values)

<!-- <p float="left">
    <img src="content/docs/wind_reconstruction_1.pdf" width="100" />
    <img src="content/docs/wind_reconstruction_2.pdf" width="100" />
</p> -->

In [ ]:
# import scipy as sp
# from scipy import signal
# import csv
# import numpy as np
# from numpy import genfromtxt
# import matplotlib.pyplot as plt
# import matplotlib
# from matplotlib.pyplot import figure, show
# import pandas as pd
# from scipy.interpolate import griddata, interp1d
# from pprz_data.pprz_data import DATA
# from numpy import linalg as la
# import matplotlib as mpl